# Testing merges before turning them into functions

In [ ]:
import numpy as np
import pandas as pd
from siuba import *
from calitp import *
import intake
import data_prep
import fuzzymatcher
pd.options.display.max_rows = 250

In [ ]:
df_5311 = data_prep.load_grantprojects()
vehicles = data_prep.load_vehiclesdata()
organizations= data_prep.load_organizations_data()

In [ ]:
vehicles['agency'].nunique()

# Merge NTD with GTFS  --> m2_df Function


In [ ]:
#drop records without NTD ID.
organizations = organizations.dropna(subset=['ntd_id'])

In [ ]:
organizations.head(1)

In [ ]:
vehicles.head(1)

In [ ]:
#merging the 2 datasets together
vehicles_gtfs = pd.merge(vehicles, organizations,  how='left', on=['ntd_id'], indicator=True)

In [ ]:
vehicles_gtfs['_merge'].value_counts()

In [ ]:
vehicles_gtfs = vehicles_gtfs.drop(columns = ['_merge'])

In [ ]:
vehicles_gtfs.head(2)

# Merging Black Cat with vehicles_gtfs

In [ ]:
right_on = ["agency"]
left_on = ["organization_name"]

In [ ]:
black_cat1 = fuzzymatcher.fuzzy_left_join(df_5311, vehicles_gtfs, left_on, right_on)

In [ ]:
organizations = black_cat1[['best_match_score','organization_name','agency']]
organizations = organizations.drop_duplicates()

In [ ]:
organizations.sort_values('best_match_score')

In [ ]:
#stuff to filter out before cutting off at scores...
subset3 = ['County of Los Angeles - Department of Public Works', 'County of Sacramento Department of Transportation']

In [ ]:
#filter out a bunch of stuff all at once
black_cat2 = black_cat1[~black_cat1.organization_name.isin(subset3)]

In [ ]:
#filter out for any scores below 0.19
black_cat2 = black_cat2.loc[black_cat2['best_match_score'] > 0.19 ]

In [ ]:
#total agencies that matched over with decent scores
black_cat2['organization_name'].nunique()

# Walking back in the other 21 agencies that were cut into a seperate dataframe

In [ ]:
#getting a list of agencies with bad scores
bad_fuzzy_matches = black_cat1.loc[black_cat1['best_match_score'] < 0.19 ]

In [ ]:
bad_fuzzy_matches = bad_fuzzy_matches[['organization_name','agency', 'ntd_id']]

In [ ]:
bad_fuzzy_matches = bad_fuzzy_matches.drop_duplicates()

In [ ]:
#making a manual crosswalk
#bad_fuzzy_matches.to_csv("./bad_fuzzy_matches.csv", index = False) 

In [ ]:
#reading crosswalk 3, what I manually inputted
crosswalk3 = pd.read_excel('corrected_bad_fuzzy_match.xlsx')

In [ ]:
#crosswalk on the left, I only want a dataframe with the poorly matched agencies
unmatched = crosswalk3.merge(df_5311, on=['organization_name'],  how='left')

In [ ]:
#now grabbing vehicles and GTFS info on the left
unmatched2 = unmatched.merge(vehicles_gtfs, on=['agency','ntd_id'],  how='left')

In [ ]:
unmatched2['organization_name'].nunique()

# R Bind

In [ ]:
GTFS_Vehicles_BC = black_cat2.append(pd.DataFrame(data = unmatched2), ignore_index=True)

In [ ]:
GTFS_Vehicles_BC = GTFS_Vehicles_BC.drop(columns=['best_match_score','__id_left','__id_right','vehicle_type'])

In [ ]:
GTFS_Vehicles_BC.shape

In [ ]:
# One last check to make sure all the names are there
Names = GTFS_Vehicles_BC[['organization_name','agency','ntd_id','itp_id','gtfs_schedule_status']]
Names = Names.drop_duplicates()
Names.sort_values('organization_name')

In [ ]:
Names.isna().sum()

In [ ]:
#GTFS_Vehicles_BC.to_csv("./GTFS_Vehicles_BC.csv", index = False) 